## This notebook uses llama-stack-client to do handle the ingesting of docs and querying them. A simple ootb RAG use case

In [1]:
import asyncio
import os

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from rich.pretty import pprint
import rich
import json
import uuid
from pydantic import BaseModel
from typing import List
#from llama_stack.distribution.library_client import LlamaStackAsLibraryClient

from dotenv import load_dotenv

load_dotenv()
#BRAVE_SEARCH_API_KEY = os.environ["BRAVE_SEARCH_API_KEY"]
HOST=os.environ["HOST"]
PORT=os.environ["LLAMA_STACK_PORT"]
MODEL_NAME=os.environ["INFERENCE_MODEL"]
#TAVILY_SEARCH_API_KEY=os.environ["TAVILY_API_KEY"]

In [2]:
client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
for tool in client.tools.list() :
    print(tool)
    print('-----')
#for provider in client.providers.list() :
#    print(provider)
#    print('-----')
#vector_providers = [
#    provider for provider in client.providers.list() if provider.api == "vector_io"
#]

Tool(description='Execute code', identifier='code_interpreter', parameters=[Parameter(description='The code to execute', name='code', parameter_type='string', required=True, default=None)], provider_id='code-interpreter', provider_resource_id='code_interpreter', tool_host='distribution', toolgroup_id='builtin::code_interpreter', type='tool', metadata=None)
-----
Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', provider_resource_id='insert_into_memory', tool_host='distribution', toolgroup_id='builtin::rag', type='tool', metadata=None)
-----
Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', provider_resource_id='knowledge_search', tool_host='distribution', toolgroup_id='bu

In [3]:
from llama_stack_client.types import Document
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "datasets.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]
documents = [
    Document(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

#vector_providers = [
#    provider for provider in client.providers.list() if provider.api == "vector_io"
#]
#selected_vector_provider = vector_providers[0]
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
    #provider_id=selected_vector_provider.provider_id,
    provider_id="sqlite-vec",
)

client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

In [4]:
# First, let's come up with a couple of examples to test the agent
examples = [
    {
        "input_query": "What precision formats does torchtune support?",
        "expected_answer": "Torchtune supports two data types for precision: fp32 (full-precision) which uses 4 bytes per model and optimizer parameter, and bfloat16 (half-precision) which uses 2 bytes per model and optimizer parameter."
    },
    {
        "input_query": "What does DoRA stand for in torchtune?",
        "expected_answer": "Weight-Decomposed Low-Rank Adaptation"
    },
    {
        "input_query": "How does the CPUOffloadOptimizer reduce GPU memory usage?",
        "expected_answer": "The CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on CPU and performing optimizer steps on CPU. It can also optionally offload gradients to CPU by using offload_gradients=True"
    },
    {
        "input_query": "How do I ensure only LoRA parameters are trainable when fine-tuning?",
        "expected_answer": "You can set only LoRA parameters to trainable using torchtune's utility functions: first fetch all LoRA parameters with lora_params = get_adapter_params(lora_model), then set them as trainable with set_trainable_params(lora_model, lora_params). The LoRA recipe handles this automatically."
    }
]

In [5]:
rag_agent = Agent(
    client,
    model=MODEL_NAME,
    instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the RAG tool to answer questions.",
    tools=[{
        "name": "builtin::rag",
        "args": {"vector_db_ids": [vector_db_id]},
    }],
)

for example in examples:
    rag_session_id = rag_agent.create_session(session_name=f"rag_session_{uuid.uuid4()}")
    response = rag_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=rag_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: The provided text is a documentation for using Meta Llama3 with torchtune and the broader ecosystem. 
It covers various aspects of fine-tuning and quantizing a model, including:

1. **Fine-tuning**: The process of training a model on a specific task or dataset to optimize its performance.
2. **Quantization-aware training (QAT)**: A technique that allows for efficient training of models with quantized 
weights and activations.
3. **Int8 dynamic activation-int4 weight configuration**: A specific QAT configuration that uses int8 activations 
and int4 weights.
4. **Post-training quantization (PTQ)**: The process of converting a model from a fake-quantized model to an actual
quantized model after fine-tuning.

The documentation provides examples of how to:

* Fine-tune a model using torchtune
* Convert a fake-quantized model to an actual quantized model using PTQ
* Use the `torch.compile` function for speedups

It also mentions that torchao provides a table listing performance and accuracy results for certain models, 
including Llama2 and Llama3.

Overall, the documentation aims to provide a comprehensive guide for users who want to fine-tune and quantize their
Meta Llama3 models using torchtune and the broader ecosystem.

Question: What does DoRA stand for in torchtune?

Agent Answer: Here is the code that corresponds to the provided specification:

```python
import torch
from torch import nn
from torch.nn import Linear, RotaryPositionalEmbeddings
from torch.nn.functional import dropout
from torch.nn.utils.rnn import pack_padded_sequence

class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, bias=False):
        super(LoRALinear, self).__init__()
        self.q_proj = Linear(in_features=in_features, out_features=out_features, bias=bias)
        self.k_proj = Linear(in_features=in_features, out_features=out_features, bias=bias)
        self.v_proj = Linear(in_features=in_features, out_features=out_features, bias=bias)

    def forward(self, x):
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        return q, k, v

class MultiHeadAttention(nn.Module):
    def __init__(self, in_features, num_heads, dropout=0.0):
        super(MultiHeadAttention, self).__init__()
        self.q_proj = Linear(in_features=in_features, out_features=in_features * num_heads, bias=False)
        self.k_proj = Linear(in_features=in_features, out_features=in_features * num_heads, bias=False)
        self.v_proj = Linear(in_features=in_features, out_features=in_features * num_heads, bias=False)
        self.dropout = dropout

    def forward(self, x):
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        # Apply LoRA
        q = LoRALinear(q, in_features=q.shape[-1], out_features=q.shape[-1])(q)
        k = LoRALinear(k, in_features=k.shape[-1], out_features=k.shape[-1])(k)
        v = LoRALinear(v, in_features=v.shape[-1], out_features=v.shape[-1])(v)
        # Apply dropout
        q = self.dropout(q)
        k = self.dropout(k)
        v = self.dropout(v)
        return q, k, v

class LoraLLama2(nn.Module):
    def __init__(self, lora_attn_modules=None, apply_lora_to_mlp=False, apply_lora_to_output=False):
        super(LoraLLama2, self).__init__()
        if lora_attn_modules is None:
            lora_attn_modules = ["q_proj", "k_proj", "v_proj"]
        else:
            lora_attn_modules = lora_attn_modules
        self.layers = nn.ModuleList([MultiHeadAttention(in_features=4096, num_heads=8) for _ in range(12)])
        for i, module in enumerate(self.layers):
            if apply_lora_to_mlp and i < len(lora_attn_modules):
                setattr(self, f"lora_{lora_attn_modules}", LoRALinear(in_features=4096, out_features=4096))
            if apply_lora_to_output:
                setattr(self, f"output_{i}", Linear(in_features=4096, out_features=4096))

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            q, k, v = layer(x)
            # Apply LoRA
            q = getattr(self, f'lora_q_proj_{i}')
            k = getattr(self, f'lora_k_proj_{i}')
            v = getattr(self, f'lora_v_proj_{i}')
            q = q + dropout(q)
            k = k + dropout(k)
            v = v + dropout(v)
            # Apply output layer
            if i == 11:
                x = Linear(in_features=q.shape[-1], out_features=4096)(q) + Linear(in_features=k.shape[-1], 
out_features=4096)(k) + Linear(in_features=v.shape[-1], out_features=4096)(v)
            else:
                x = q
        return x

# Create a Llama2 model without LoRA
base_model = Llama2()

# Create a Llama2 model with LoRA applied to some layers
lora_model = LoraLLama2(lora_attn_modules=["q_proj", "v_proj"])
```

This code defines the `LoRALinear` class, which represents a linear layer with LoRA. It also defines the 
`MultiHeadAttention` class, which is a multi-head attention mechanism that applies LoRA to its input. The 
`LoraLLama2` class is a Llama2 model that can be configured to apply LoRA to some of its layers.

The code then creates two models: one without LoRA and one with LoRA applied to the query and value projections in 
each self-attention layer.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: []

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: Here is the code that corresponds to the provided specification:

```python
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from peft.peft_utils import get_adapter_params, set_trainable_params

# Load the pre-trained model and tokenizer
model_name = "facebook/llamalarge-lm"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up the dataset and data loader
class MyDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts
        inputs = self.tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        return inputs

dataset = MyDataset(["text1", "text2"], tokenizer)
data_loader = DataLoader(dataset, batch_size=32)

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)

# Fine-tune the model with LoRA
def fine_tune_lora(model, data_loader, optimizer, scheduler):
    for epoch in range(1):
        for batch in data_loader:
            inputs = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            optimizer.zero_grad()
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

# Load the pre-trained weights and set only LoRA parameters to trainable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
lora_model = PeftModel(model, device=device)

base_model_state_dict = model.state_dict()
lora_model.load_state_dict(base_model_state_dict, strict=False)

# Set only LoRA parameters to trainable
lora_params = get_adapter_params(lora_model)
set_trainable_params(lora_model, lora_params)

# Print the total number of parameters and trainable parameters
total_params = sum()
trainable_params = sum()
print(f"{total_params} total params, {trainable_params} trainable params, {(100.0 * trainable_params / 
total_params):.2f}% of all params are trainable.")
```

Note that this code assumes you have the `peft` library installed and have replaced the `model_name`, `tokenizer`, 
and `dataset` variables with your own values. Additionally, you may need to modify the `fine_tune_lora` function to
suit your specific use case.